## EigenSNP-WAS - Gene-level Association Function
## Steven D. Shaw

If you would like to use this function, please feel free to do so, and give credit if it is due. I am happy to answer any questions regarding this function, or, if you would like to work together on a project related to gene-level association, please contact me. 

Email: shawsd@umich.edu

Current status: The function is in full working order, and outputs a .txt file with one gene assocation per DV on each line with the following variables, gene name, number of SNPs observed in the gene, number of eigenSNPs within the gene, association statistics (F-value, sum of squares, p-value), and DV (as multiple DVs can be tested at once). To get the function working properly with your own data, ensure that behavioral/ survey/ phenotypic data is formated properly (see comments throughout code) and genetic data has been pre-cleaned in PLINK (e.g., MAF, MIND, etc.). 

I am in the process of optimizing the speed of the function. Currently, with a dataset of approx. 15k individuals, hg18 gene set (approx. 20k genes), and 4 phenotypes, it takes 4.5 hours to run to completion.

If you happen to notice any errors, please do let me know - we all make mistakes and can help eachother overcome and learn from our mistakes :)

## Import packages

In [ ]:
import pandas as pd
import numpy as np
from pandas_plink import read_plink1_bin
from os.path import join
from os import path
from pandas_plink import get_data_folder
from pandas_plink import read_plink
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime

import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.api import add_constant
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

import time
import matplotlib.pyplot as plt

## DATA LOADING & MERGING

## Genetic Data

In [ ]:
# Load in genetic data
    # QC PLINK vars: --mind 0.05 --geno 0.01 --maf 0.01
    # Rename SNPs: --update-map SNP_kgpID2rsID.txt --update-name

# Insert name of genetic data file - should be precleaned in PLINK
gene_data_filename = ''
    
(bim, fam, bed) = read_plink(gene_data_filename, verbose=False)
bim.set_index('chrom')
fam.set_index('i')
bim = bim.sort_values(by=['chrom', 'pos'])

bim.to_csv("bim.csv")

########

# Provide a genome gene set list (E.g., 'glist-hg18.txt')
gene_set_filename = ''

# List of genes in the human genome, and their location (chromosome, start, and end)
hg_18 = pd.read_csv(gene_set_filename, 
                 sep=' ', 
                 header=None, 
                 names=["chromosome", "Start", "End", "gene"])

# add column that is current position just in case for joining afterwards
hg_18['row_num'] = np.arange(len(hg_18))

hg_18 = hg_18.sort_values(by=['Start', 'End'])
hg_18

## MAP SNPs TO GENES

In [ ]:
# Reorganize the genetic data (bim) to determine every gene that each SNP is located on
# Use gene location information in glist-hg19.txt (dataset: hg_18) to traverse through chromosomes, sequentially,
# gene-by-gene and identify which SNPs are in each gene

hg_18.set_index('chromosome')

snp_to_gene_mapping = []
for i in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 
          '11', '12', '13', '14', '15', '16', '17', '18', '19',
          '20', '21', '22']:
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    print("chromosome: ", i)
    chrom_x = bim.query("chrom=='{0}'".format(i))
    hg_x = hg_18.query("chromosome=='{0}'".format(i))
    start = hg_x['Start'].min()
    end = hg_x['End'].max()
    chrom_x = chrom_x[chrom_x["pos"].between(start, end)]
    length = len(chrom_x)

    j = 0
    
    for ind, r in hg_x.iterrows():
        while chrom_x.iloc[j, 3] < r['Start']:
            j += 1
            if j >= length:
                break
        if j >= length:
                break
        while chrom_x.iloc[j, 3] <= r['End']:
            snp_to_gene_mapping.append([chrom_x.iloc[j, 0], chrom_x.iloc[j, 1], chrom_x.iloc[j, 2], 
                                chrom_x.iloc[j, 3], chrom_x.iloc[j, 4], chrom_x.iloc[j, 5],
                                chrom_x.iloc[j, 6], r['Start'], r['End'], r['gene']])
            j += 1
            if j >= length:
                break
        if j >= length:
                break

In [ ]:
#Organize traversal into dataframe
bim_with_genes = pd.DataFrame(snp_to_gene_mapping, columns = ['chrom' , 'snp', 'cm', 'pos', 
                                                               'a0', 'a1', 'i','start', 'end', 'gene'])
print(bim_with_genes)

# Save this file, so you don't need to rerun above code on other computers
# bim_with_genes.to_csv("bim_with_genes.csv")

In [ ]:
# Load in bim_with_genes for EigenSNP function
#bim_with_genes = pd.read_csv('bim_with_genes.csv')
#print(bim_with_genes)

## EIGENSNP-WAS FUNCTION

### Required Functions

PCA NOTES:

Takes in a df where each row is a participant and each column is a SNP. Data input should be minor allele dosage.

PCA is run once per gene in main function.

It will output a df where each row is a participant and each column contains the eigenvectors from the PCA.

The number of PCs is determined by the cummulative variance required set by the 'limit' in the function (typically, greater than 90% variance explained).


REGRESION NOTES:

Takes in two dfs:

i_df : a df where rows are participants and columns are the independent variables

t_df : a df where rows are participants and column is the target dependent variable (i.e. BMI)

BOTH DATAFRAMES SHOULD HAVE SAME PARTICIPANT ORDER

In [ ]:
##PCA Functions
def PC_eigen_vals(gene_df):
    X = StandardScaler().fit_transform(gene_df)
    limit = 0.95

    #perform PCA
    pca = decomposition.PCA(n_components=len(gene_df.columns))
    #print(pca.fit(X))
    pca.fit(X)
    X = pca.transform(X)
    
    #extract variance for each PC
    var = pca.explained_variance_ratio_
    
    # find num PCs
    #np.cumsum(var)
    
    # find num PCs
    cum_var = 0
    i = 0
    while cum_var <= limit and i < len(var):
        cum_var += var[i]
        i += 1
    
    # separate eigen vect columns based on limit
    X_PCs = X[:,0:i]
    
    
    # add to dictionary
    PCs = {}
    for i in range(0,X_PCs.shape[1]):
        PCs["PC{0}".format(i)] = pd.Series(X[:,i:i+1].transpose()[0])
    
    # convert to df
    PC_df = pd.DataFrame(PCs)
    
    # Returns pd df
    return PC_df

## Regression Functions

# takes input df and target df
def OLS_mod(i_df, t_df):
    # set dep and ind variables
    X = i_df[[i for i in i_df.columns]]
    y = t_df[[i for i in t_df.columns]]

    # add constant (intercepts)
    X = sm.add_constant(X)

    # least squares model
    mod = sm.OLS(y, X).fit()
    #print(mod.summary())

    return mod

In [ ]:
#FUNCTION

# This loop takes each gene that we observe, obtained only the genetic dosage data that for that gene from the main
# genetic dataset, then:
    # conducts PCA on SNPs within that gene, and retains eigenvectors from the number of PCs 
        # that explain >90% variance (i.e., eigenSNPs)
    # merges the eigenSNPs with survey/non-genetic data for use with regression
    # runs treatment and control regression models and compares these models to get the unique effect of the gene
    # outputs the number of SNPs observed in the gene, the number of eigenSNPs, and the p-value

#SURVEY DATA
    #Survey data must contain continuous dependent variables (variable names inserted into list below),
    #participant ID as 'IID', 'Age', and principle components from a genome wide PCA, labeled 
    #'EV1', 'EV2', 'EV3', 'EV4', etc. 
    
EigenSNPWAS_results = pd.DataFrame(columns=['Gene', 'SNPs', 'EigenSNPs', 'ss_diff', 'f_stat','p_val','DV'])

#Chromosome counter so we can stop the loop, for testing
chrom_counter = 0

bim_with_genes.set_index('chrom')

#Get a list of chromosomes in genetic dataset
chrom_x = bim_with_genes['chrom'].unique()

# List the names of your DVs:
DV_list = ['']

#Current time
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

#Start timer
start_time = time.time()

for j in chrom_x:
    
    time_chk_chrom_loop = time.time()
    
#     # Specify how many chromosomes to run - remove to run all
#     if chrom_counter == 2:
#         break
    
    print('Chromosome:', j)
    
    chrom_df = bim_with_genes.query("chrom=='{0}'".format(j))
    #Create list of observed genes
    chrom_df.set_index('gene')
    genes_in_chrom_x = chrom_df['gene'].unique()
    
    chrom_counter += 1
    
    #Chromosome counter so we can stop the loop, for testing
    gene_counter = 0
#t1
    time_chk_1 = time.time()
    
    for i in genes_in_chrom_x:
        
        time_chk_gene_loop = time.time()
        
#         # Specify how many genes to run - remove to run all
#         if gene_counter == 5:
#             break

        #Display progress bar
            #Change division size for different progress indicators
        if gene_counter in list(range(1, len(genes_in_chrom_x), 500)): 
            print('Gene:', gene_counter, 'of', len(genes_in_chrom_x))
            elapsed_time = time.time() - start_time
            print("Elapsed Time =", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

        # Get the SNP data from bed file
        gene_df = chrom_df.query("gene=='{0}'".format(i))
        x = bed[gene_df.i.values, :].compute() #x contains only dosage data for SNPs in current gene

        # transpose so that data are formatted correctly (columns are SNPs, rows are participants)
        x_trans = x.transpose()
        #print(x_trans)

        # Get SNP names, create dataframe with SNP labels (as columns) for use in PCA
        snps = gene_df['snp']
        SNP_data_for_PC = pd.DataFrame(x_trans, columns = snps)
        #print('with NaN:', SNP_data_for_PC)

        #Replace NaNs with mean for each column
        SNP_data_for_PC.fillna(SNP_data_for_PC.mean(), axis=0, inplace=True)

        #There are some strange columns of all NaNs, remove these first
        SNP_data_for_PC.dropna(axis=0, how='all', inplace=True)

        #finally, if any NAs remain within a row, drop that row
            #very few of these cases, but needed to keep function running (e.g., CSMD1 in chr8, has random missingness)
        SNP_data_for_PC.dropna(axis=1, how='any', inplace=True)
        #print(SNP_data_for_PC.shape)
        #print('without NaN:', SNP_data_for_PC)

        #Count number of SNPs
        SNP_count = len(SNP_data_for_PC.columns)
        #print(i)
        #print(SNP_count)
#t2 
        time_chk_2 = time.time()
        
        try:
            if SNP_count == 0:
                #EigenSNPWAS.append([i, 0, np.nan, np.nan])
                EigenSNPWAS_results = EigenSNPWAS_results.append({'Gene': i,
                                'SNPs': 0,
                                'EigenSNPs': np.nan,
                                'ss_diff': np.nan,
                                'f_stat': np.nan,
                                'p_val': np.nan,
                                'DV': np.nan},                                
                                ignore_index=True)
            else:
                if SNP_count == 2:
                    PCA = SNP_data_for_PC
                else:
                    #Conduct PCA - output contains dataframe with EigenSNPs (cut to >90% threshold)
                    PCA = PC_eigen_vals(SNP_data_for_PC)
                    #print(PCA)

                #Count number of EigenSNPs
                PC_Eigen_count = len(PCA.columns)
                #print(PC_Eigen_count)

                # Take output of PCA - eigenSNPs - and merge with survey/non-genetic data
                #Get ID codes for gene data - use key values (i) from query with gene name
                    #gene_df contains only the SNPs we are interested in

                #Append gene IDs to PCA output
                PCA['IID'] = pd.to_numeric(fam['iid'], errors="coerce")
                # new code - Add Sex variable from fam file
                PCA['Sex'] = pd.to_numeric(fam['gender'], errors="coerce")
                #print('fam iid_type:', PCA['IID'].dtypes)
                #print(PCA)

                #merge eigenSNPs with dataset: DATA_survey 
                PCA = PCA[PCA['IID'].notna()]
#t3                
                time_chk_3 = time.time()
                
                for DV in DV_list:
                    DATA_survey_DV = DATA_survey[[DV, 'IID', 'Age', 'EV1', 'EV2', 'EV3', 'EV4']]
                    DATA_all_for_gene = pd.merge(DATA_survey_DV, PCA, left_on='IID', right_on='IID', how='inner')
                    #print(list(DATA_all_for_gene.columns)) #View columns in dataset

                    #Drop the IID column, as it is no longer needed
                    DATA_all_for_gene.drop('IID', axis=1, inplace=True)
                    #Remove any rows with missing values
                    DATA_all_for_gene.dropna(inplace=True)
                    DATA_DV_only = pd.DataFrame(DATA_all_for_gene[DV])

                    #Create treatment dataset
                    DATA_treatment_model = DATA_all_for_gene.drop(DV, axis=1)
                    #print(DATA_treatment_model)

                    #Create control dataset
                    DATA_control_model = DATA_all_for_gene[['Age', 'Sex', 'EV1', 'EV2', 'EV3', 'EV4']]
                    #print(DATA_control_model)

                    # TREATMENT Model
                    treat_mod = OLS_mod(DATA_treatment_model, DATA_DV_only)     ## controls + EigenSNPs
                    #treat_mod.summary()

                    # CONTROL Model
                    cont_mod = OLS_mod(DATA_control_model, DATA_DV_only)   ## controls only
                    #cont_mod.summary()

                    #Conduct ANOVA to compare TREATMENT and CONTROL models
                    aov_table = sm.stats.anova_lm(cont_mod, treat_mod, typ=1)
                    #print(aov_table)

                    # Extract stats from model (F and p-val)
                    gene_ss_diff = aov_table['ss_diff'][1]
                    gene_fstat = aov_table['F'][1]
                    gene_pval = aov_table['Pr(>F)'][1]
                    #print('f:', gene_fstat, 'p:', gene_pval)

                    #print('Gene:', i, 'SNPs:', SNP_count, 'EigenSNPs:', PC_Eigen_count, 'p_val:', gene_pval)

                    #EigenSNPWAS.append([i, SNP_count, PC_Eigen_count, gene_pval])
                    EigenSNPWAS_results = EigenSNPWAS_results.append({'Gene': i,
                                                                    'SNPs': SNP_count,
                                                                    'EigenSNPs': PC_Eigen_count,
                                                                    'ss_diff': gene_ss_diff,
                                                                    'f_stat': gene_fstat,
                                                                    'p_val': gene_pval,
                                                                    'DV': DV},
                                                                    ignore_index=True)
        except ValueError:  #happens once in chromosome 8 (dealt with, but will leave try/except just in case)
            EigenSNPWAS_results = EigenSNPWAS_results.append({'Gene': i,
                                                            'SNPs': SNP_count,
                                                            'EigenSNPs': np.nan,
                                                            'ss_diff': np.nan,
                                                            'f_stat': np.nan,
                                                            'p_val': np.nan,
                                                            'DV': np.nan},                                
                                                            ignore_index=True)
#t4
        time_chk_4 = time.time()
        gene_counter += 1
#         print('Time_checks:', '1:', time_chk_1-time_chk_chrom_loop, '2:', time_chk_2-time_chk_gene_loop,
#              '3:', time_chk_3-time_chk_2, '4:', time_chk_4-time_chk_3)